---
title: "Quiz_4"
author: "Gabin Lee"
date: "2023-05-21"
categories: [new, code, analysis]
image: "image.png"
---

**This notebook is an exercise in the [Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/manipulating-geospatial-data).**

---


# 소개

당신은 스타벅스의 빅 데이터 분석가입니다 ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5). 에 다음 매장을 찾고자 합니다. 이 로스터리는 일반 스타벅스 매장보다 훨씬 크며, 다양한 음식과 와인 옵션, 고급스러운 라운지 공간 등 여러 가지 추가 기능을 갖추고 있습니다. 캘리포니아 주의 다양한 군집의 인구 통계를 조사하여 잠재적으로 적합한 위치를 결정하게 될 것입니다.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BIyE6kR.png" width="450"><br/><br/>
</center>

시작하기 전에, 아래의 코드 셀을 실행하여 모든 설정을 완료하세요.

In [18]:
#!pip install geopy

     -------------------------------------- 119.8/119.8 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00


In [34]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim            # What you'd normally run
# from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

# from learntools.core import binder
# binder.bind(globals())
# from learntools.geospatial.ex4 import *

이전 연습에서 사용한 `embed_map()` 함수를 사용하여 지도를 시각화할 것입니다.

In [35]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# 연습문제
### 1) 누락된 위치 정보를 지오코딩하십시오.

다음 코드 셀을 실행하여 캘리포니아 주에 있는 스타벅스 위치를 포함한 DataFrame `starbucks`를 생성합니다.

In [36]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("archive_4/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


대부분의 매장은 알려진 (위도, 경도) 위치를 가지고 있습니다. 하지만 버클리 시의 모든 위치는 누락되었습니다.

In [37]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


아래의 코드 셀을 사용하여 Nominatim 지오코더를 사용하여 이러한 값을 채워 넣을 수 있습니다.

참고로, 튜토리얼에서는 `geopy.geocoders`의 `Nominatim()`을 사용하여 값을 지오코딩하는 방법을 알아보았습니다. 이 방법은 이 강의 외부에서 본인의 프로젝트에서도 사용할 수 있습니다. 

이 연습에서는 약간 다른 함수인 `learntools.geospatial.tools`의 `Nominatim()` 함수를 사용할 것입니다. 이 함수는 노트북 상단에 임포트되었으며 GeoPandas의 함수와 동일하게 작동합니다.

다시 말해, 다음 조건을 만족한다면:
- 노트북 상단의 임포트 문을 변경하지 않고,
- 아래 코드 셀에서 지오코딩 함수를 `geocode()`로 호출한다면,

원하는 대로 코드가 작동할 것입니다!

In [38]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def geocode(address):
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

# Check your answer
# q_1.check()

In [ ]:
# Line below will give you solution code
#q_1.solution()

### 2) 버클리 위치 확인하기

방금 찾은 버클리의 (위도, 경도) 위치를 확인해봅시다. OpenStreetMap 스타일로 시각화해보세요.

In [43]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
berkeley_starbucks = starbucks[starbucks["City"] == "Berkeley"].dropna(subset=["Latitude", "Longitude"])

for idx, row in berkeley_starbucks.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(m_2)
# Uncomment to see a hint
#q_2.a.hint()

# Show the map
# embed_map(m_2, 'q_2.html')
m_2

In [ ]:
# Get credit for your work after you have created a map
# q_2.a.check()

# Uncomment to see our solution (your code may look different!)
#q_2.a.solution()

버클리에 있는 다섯 개의 위치 중에서 (위도, 경도) 위치가 올바른 도시에 위치한 것으로 보이는 위치는 몇 개인가요?

In [ ]:
# View the solution (Run this code cell to receive credit!)
# q_2.b.solution()

### 3) 데이터를 통합하세요.

아래의 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱 킬로미터), 그리고 고유한 ID("GEOID" 열)가 포함된 GeoDataFrame `CA_counties`를 로드하세요. "geometry" 열에는 카운티 경계를 나타내는 다각형이 포함되어 있습니다.

In [44]:
CA_counties = gpd.read_file("archive_4/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

c:\Users\leegabin\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


다음으로, 세 개의 데이터프레임을 생성합니다:
- `CA_pop`은 각 카운티의 인구 추정치를 포함합니다.
- `CA_high_earners`는 연간 수입이 15만 달러 이상인 가구의 수를 포함합니다.
- `CA_median_age`는 각 카운티의 중간 연령을 포함합니다.

In [45]:
CA_pop = pd.read_csv("archive_4/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("archive_4/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("archive_4/CA_county_median_age.csv", index_col="GEOID")

다음 코드 셀을 사용하여 `CA_counties` GeoDataFrame을 `CA_pop`, `CA_high_earners`, `CA_median_age`와 결합합니다.

결과 GeoDataFrame의 이름을 `CA_stats`로 지정하고, "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", "median_age"의 7개 열을 가져야 합니다.

In [47]:
# Your code here
CA_stats = CA_counties.merge(CA_pop[['population']], left_on='GEOID', right_index=True)
CA_stats = CA_stats.merge(CA_high_earners[['high_earners']], left_on='GEOID', right_index=True)
CA_stats = CA_stats.merge(CA_median_age[['median_age']], left_on='GEOID', right_index=True)
CA_stats = CA_stats[['GEOID', 'name', 'area_sqkm', 'geometry', 'population', 'high_earners', 'median_age']]

# Check your answer
# q_3.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

이제 모든 데이터를 한 곳에 모았으므로, 여러 열의 조합을 사용하여 통계를 계산하는 것이 훨씬 쉬워졌습니다. 다음 코드 셀을 실행하여 "density" 열을 생성하여 인구 밀도를 계산하세요.

In [48]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) 어떤 군도가 유망해 보이나요?

모든 정보를 하나의 GeoDataFrame으로 통합하면 특정 기준을 충족하는 군도를 선택하는 것도 훨씬 쉬워집니다.

다음 코드 셀을 사용하여 `CA_stats` GeoDataFrame의 일부 행(및 모든 열)을 포함하는 GeoDataFrame `sel_counties`를 만드세요. 특히 다음 기준을 충족하는 군도를 선택해야 합니다:
- 연간 $150,000 이상의 소득을 가진 가구가 적어도 100,000개 이상 있는 경우,
- 중간 연령이 38.5보다 작은 경우, 그리고
- 주민의 밀도가 285 이상인 경우 (제곱 킬로미터당).

또한, 선택된 군도는 다음 기준 중 하나 이상을 충족해야 합니다:
- 연간 $150,000 이상의 소득을 가진 가구가 적어도 500,000개 이상 있는 경우,
- 중간 연령이 35.5보다 작은 경우, 또는
- 주민의 밀도가 1400 이상인 경우 (제곱 킬로미터당).

In [49]:
# Your code here
sel_counties = CA_stats[
    (CA_stats['high_earners'] >= 100000) &
    (CA_stats['median_age'] < 38.5) &
    (CA_stats['population'] / CA_stats['area_sqkm'] >= 285) &
    (
        (CA_stats['high_earners'] >= 500000) |
        (CA_stats['median_age'] < 35.5) |
        (CA_stats['population'] / CA_stats['area_sqkm'] >= 1400)
    )
]
sel_counties

# Check your answer
# q_4.check()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


In [ ]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

### 5) 식별한 매장의 수는 몇 개인가요?

다음 Starbucks Reserve Roastery 위치를 찾을 때, 선택한 군도 내에 있는 모든 매장을 고려하고 싶습니다. 그러면 선택한 군도 내에 몇 개의 매장이 있는지 알 수 있을까요?

이 질문에 대답하기 위해, 다음 코드 셀을 실행하여 모든 스타벅스 위치를 포함하는 GeoDataFrame `starbucks_gdf`를 생성하세요.

In [50]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

c:\Users\leegabin\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


선택한 군도에는 몇 개의 매장이 있나요?

In [53]:
# Fill in your answer
selected_counties_stores = gpd.sjoin(starbucks_gdf, sel_counties, op='within')
num_stores = len(selected_counties_stores)
num_stores

# Check your answer
# q_5.check()

c:\Users\leegabin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


1043

In [ ]:
# Lines below will give you a hint or solution code
#q_5.hint()
#q_5.solution()

### 6) 매장 위치를 시각화하는 지도를 생성해보겠습니다.

이전 질문에서 확인한 매장의 위치를 보여주는 지도를 생성해보겠습니다.

In [54]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
for idx, row in selected_counties_stores.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_6)

m_6
# Uncomment to see a hint
#q_6.hint()

# Show the map
# embed_map(m_6, 'q_6.html')

In [ ]:
# Get credit for your work after you have created a map
# q_6.check()

# Uncomment to see our solution (your code may look different!)
#q_6.solution()

# Keep going

Learn about how **[proximity analysis](https://www.kaggle.com/alexisbcook/proximity-analysis)** can help you to understand the relationships between points on a map.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*